In [ ]:
import os

import tensorflow as tf
from tensorflow.keras import layers, Model


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150,150,3),
                               include_top = False,
                               weights = None)

In [ ]:
pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed10')
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation = 'relu')(x)

x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation = 'sigmoid')(x)

In [ ]:
model = Model(pre_trained_model.input, x)

In [ ]:
model.compile(optimizer = RMSprop(lr=0.0001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [ ]:
# import zipfile
# local_zip = 'tmp\cats_and_dogs_filtered.zip'

# zip_ref = zipfile.ZipFile(local_zip, 'r')

# zip_ref.extractall('tmp\Inception')
# zip_ref.close()

In [ ]:
base_dir = 'tmp\Inception\cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

In [ ]:
history = model.fit_generator(
                    train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 100,
                    epochs = 5,
                    validation_steps = 50,
                    verbose = 1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()